# Advent of Code 2015

My solutions for [Advent of Code 2015](https://adventofcode.com/2015), based on Norvig [notebook](https://github.com/norvig/pytudes/blob/master/ipynb/Advent-2020.ipynb)

# Day 0: Imports and Utility Functions

Preparations prior to Day 1:
- Some imports.
- A way to read the day's data file and to print/check the output.
- Some utilities that are likely to be useful.

In [92]:
from itertools   import chain, combinations, tee, filterfalse, permutations, groupby
from functools   import reduce, cache
from hashlib     import md5
from typing      import Tuple, List, Set, Union, Dict
from contextlib  import contextmanager

import json
import operator
import re
import numpy as np

In [2]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    with open(f'data/input-{day}') as f:
        sections = f.read().rstrip().split(sep)
        return list(map(parser, sections))

def do(day, *answers) -> List[int]:
    "E.g., do(3) returns [day3_1(in3), day3_2(in3)]. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g:
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part:
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
        else:
            got.append(None)
    return got

In [3]:
Number = Union[float, int]
Atom = Union[Number, str]
Char = str # Type used to indicate a single character

cat = ''.join
flatten = chain.from_iterable

def partition(pred, iterable):
    "Use a predicate to partition entries into false entries and true entries"
    # partition(is_odd, range(10)) --> 0 2 4 6 8   and  1 3 5 7 9
    t1, t2 = tee(iterable)
    return filterfalse(pred, t1), filter(pred, t2)

def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)

def prod(numbers) -> Number:
    "The product of an iterable of numbers."
    return reduce(operator.mul, numbers, 1)

def dot(A, B) -> Number:
    "The dot product of two vectors of numbers."
    return sum(a * b for a, b in zip(A, B))

def ints(text: str) -> Tuple[int]:
    "Return a tuple of all the integers in text."
    return mapt(int, re.findall('-?[0-9]+', text))

def lines(text: str) -> List[str]:
    "Split the text into a list of lines."
    return text.strip().splitlines()

def mapt(fn, *args):
    "Do map(fn, *args) and make the result a tuple."
    return tuple(map(fn, *args))

def atoms(text: str, ignore=r'', sep=None) -> Tuple[Union[int, str]]:
    "Parse text into atoms separated by sep, with regex ignored."
    text = re.sub(ignore, '', text)
    return mapt(atom, text.split(sep))

def atom(text: str, types=(int, str)):
    "Parse text into one of the given types."
    for typ in types:
        try:
            return typ(text)
        except ValueError:
            pass

@contextmanager
def binding(**kwds):
    "Bind global variables within a context; revert to old values on exit."
    old_values = {k: globals()[k] for k in kwds}
    try:
        globals().update(kwds)
        yield # Stuff within the context gets run here.
    finally:
        globals().update(old_values)

Notes:
-  Since I'm not even attempting to compete to be among the first 100 people to find a solution, I'll take the time to write docstrings; to use reasonable variable names (not single-letter names); and to give type annotations for most functions (but not the day functions, which all return int, except day21_2).
-  Traditionally, a lot of AoC problems are solved by one of the following two forms:
  - quantify(inputs, P): How many of your input items have property P?
  - sum(map(F, inputs)): What is the sum of the result of applying F to each input item?
- Some days I will re-use code that was defined on a previous day.
- I will give a few tests using assert, but far fewer test cases than if I was programming seriously.

# Day 1: Not Quite Lisp
1. Find on which floor is Santa
2. Find when Santa enter the basement

In [4]:
in1: str = cat(data(1))

In [5]:
def day1_1(parentheses):
    """ Find on which floor is Santa """
    return parentheses.count("(") - parentheses.count(")")

In [6]:
def day1_2(parentheses):
    """ Find when Santa enter the basement """
    level = 0
    for i, parenthese in enumerate(parentheses):
        level += 1 if parenthese == "(" else -1
        if level == -1:
            return i + 1

In [7]:
do(1, 74, 1795)

[74, 1795]

# Day 2: I Was Told There Would Be No Math
1. How many square feet of wrapping paper ?
2. How many feet of ribbon ?

In [8]:
Dimensions = Tuple[int, int, int]

def parse_dimensions(line: str):
    return tuple(int(dimension) for dimension in line.split("x"))

in2: List[Dimensions] = data(2, parse_dimensions)

In [9]:
def day2_1(dimensions):
    """ How many square feet of wrapping paper ? """
    paper = 0
    for xyz in dimensions:
        sides = [s1 * s2 for s1, s2 in combinations(xyz, 2)]
        paper += sum(side * 2 for side in sides)
        paper += min(sides)
    return paper

In [10]:
def day2_2(dimensions):
    """ How many feet of ribbon ? """
    ribbon = 0
    for xyz in dimensions:
        xyz = sorted(xyz)
        ribbon += sum(xyz[:2]) * 2
        ribbon += prod(xyz)
    return ribbon

In [11]:
do(2, 1606483, 3842356)

[1606483, 3842356]

# Day 3: Perfectly Spherical Houses in a Vacuum
1. How many houses delivered by Santa ?
2. How many houses delivered by Santa and Robo-Santa ?

In [12]:
Directions = List[Char]
Coordinates = Tuple[int, int]

in3: Directions = cat(data(3))

In [13]:
def day3_1(directions):
    """ How many houses delivered by Santa ? """
    return len(deliver(directions))

def deliver(directions: Directions) -> Set[Coordinates]:
    cx, cy = 0, 0
    houses = {(cx, cy)}
    for direction in directions:
        if direction == ">":
            cx += 1
        elif direction == "<":
            cx -= 1
        elif direction == "^":
            cy += 1
        elif direction == "v":
            cy -= 1
        houses.add((cx, cy))
    return houses

In [14]:
def day3_2(directions):
    """ How many houses delivered by Santa and Robo-Santa ? """
    return len(deliver(directions[::2]) | deliver(directions[1::2]))

In [15]:
do(3, 2572, 2631)

[2572, 2631]

# Day 4: The Ideal Stocking Stuffer
1. Find md5 that starts with 5 zeroes.
2. Find md5 that starts with 6 zeroes.

In [16]:
in4: str = "ckczppom"

In [17]:
def day4_1(secret_key, n=5):
    i = 1
    while True:
        md5_value = md5(bytes(secret_key + str(i), encoding="utf8")).hexdigest()
        if md5_value.startswith("0"*n):
            return i
        i += 1

In [18]:
def day4_2(secret_key):
    return day4_1(secret_key, n=6)

In [19]:
do(4, 117946, 3938038)

[117946, 3938038]

# Day 5: Doesn't He Have Intern-Elves For This?
1. Count valid words.
2. Count valid words again.

In [20]:
in5 = data(5)

In [21]:
def day5_1(words):
    """ Count valid words """
    return quantify(words, validate1)

def validate1(word):
    vowels = len(re.findall("[aeiou]", word)) >= 3
    bad = re.search("ab|cd|pq|xy", word) is not None
    double = re.search("(?P<letter>.)(?P=letter)", word) is not None
    return vowels and not bad and double

In [22]:
def day5_2(words):
    """ Count valid words again """
    return quantify(words, validate2)

def validate2(word):
    twice = re.search("(?P<pair>..).*(?P=pair)", word) is not None
    repeat_between = re.search("(?P<letter>.).(?P=letter)", word) is not None
    return twice and repeat_between

In [23]:
do(5, 255, 55)

[255, 55]

# Day 6: Probably a Fire Hazard
1. How many lights are lit ?
2. What is the total brightness of all lights combined ?

In [24]:
Instruction = Tuple[str, int, int, int, int]

def instruction_parser(line):
    inst, x1, y1, x2, y2 = re.fullmatch(
        "(toggle|turn on|turn off) (\d+),(\d+) through (\d+),(\d+)",
        line,
    ).groups()
    return inst, int(x1), int(y1), int(x2), int(y2)

in6 = data(6, instruction_parser)

In [25]:
def day6_1(instructions):
    """ How many lights are lit ? """
    lights = np.full((1000, 1000), False)
    for inst, x1, y1, x2, y2 in instructions:
        if inst == "toggle":
            lights[x1:x2+1,y1:y2+1] = ~lights[x1:x2+1,y1:y2+1]
        if inst == "turn on":
            lights[x1:x2+1,y1:y2+1] = True
        if inst == "turn off":
            lights[x1:x2+1,y1:y2+1] = False
    return np.count_nonzero(lights)

In [26]:
def day6_2(instructions):
    """ What is the total brightness of all lights combined ? """
    lights = np.full((1000, 1000), 0)
    for inst, x1, y1, x2, y2 in instructions:
        if inst == "toggle":
            lights[x1:x2+1,y1:y2+1] += 2
        if inst == "turn on":
            lights[x1:x2+1,y1:y2+1] += 1
        if inst == "turn off":
            lights[x1:x2+1,y1:y2+1] -= 1
            lights[x1:x2+1,y1:y2+1] = lights[x1:x2+1,y1:y2+1].clip(min=0)
    return np.sum(lights)

In [27]:
do(6, 543903, 14687245)

[543903, 14687245]

# Day 7: Some Assembly Required
1. What signal is ultimately provided to wire a ?
2. Override B and what signal is ultimately provided to wire a ?

In [28]:
Instruction = Dict[str, List[str]]

def instruction_parser(line):
    left, right = line.split(" -> ")
    return right, left.split(" ")

in7 = dict(data(7, instruction_parser))

In [29]:
def day7_1(instructions):
    """ What signal is ultimately provided to wire a ? """
    @cache
    def compute(var):
        if var.isdigit():
            return int(var)
        instruction = instructions[var]
        if len(instruction) == 1:
            return compute(instruction[0])
        elif len(instruction) == 2:
            op, right = instruction
            if op == "NOT":
                return ~ compute(right)
        elif len(instruction) == 3:
            left, op, right = instruction
            if op == "AND":
                return compute(left) & compute(right)
            if op == "OR":
                return compute(left) | compute(right)
            if op == "LSHIFT":
                return compute(left) << compute(right)
            if op == "RSHIFT":
                return compute(left) >> compute(right)
        raise Exception(var, instruction)
    return compute("a")

In [30]:
def day7_2(instructions):
    """ Override B and what signal is ultimately provided to wire a ? """
    a = day7_1(instructions)
    instructions["b"] = [str(a)]
    return day7_1(instructions)

In [31]:
do(7, 956, 40149)

[956, 40149]

# Day 8: Matchsticks
1. Number of characters in memory
2. Number of charecters escaped

In [32]:
in8: List[str] = data(8)

In [33]:
def day8_1(strings):
    """ Number of characters in memory """
    string_len = sum(map(len, strings))
    memory_len = sum(len(eval(string)) for string in strings)
    return string_len - memory_len

In [34]:
def day8_2(strings):
    """ Number of characters escaped """
    res = 0
    for string in strings:
        res += 2
        res += string.count('"')
        res += string.count("\\")
    return res

In [35]:
do(8, 1371, 2117)

[1371, 2117]

# Day 9: All in a Single Night
1. Find the shortest path.
2. Find the longest path.

In [36]:
Distance = Tuple[str, str, int]

def distances_parser(line):
    src, dst, dist = re.fullmatch("(.+) to (.+) = (.+)", line).groups()
    return src, dst, int(dist)

in9: List[Distance] = data(9, distances_parser)

In [37]:
def day9_1(distances):
    """ Find the shortest path """
    routes = compute_routes(distances)
    return min(routes.values())

def compute_routes(distances):
    towns = set()
    distances_dict = {}
    for src, dst, dist in distances:
        distances_dict[(src, dst)] = dist
        distances_dict[(dst, src)] = dist
        towns.add(src)
        towns.add(dst)

    results = {}
    for permutation in permutations(towns, len(towns)):
        results[permutation] = sum(
            distances_dict[(src, dst)]
            for src, dst in zip(permutation, permutation[1:])
        )
    return results

In [38]:
def day9_2(distances):
    """ Find the longest path """
    routes = compute_routes(distances)
    return max(routes.values())

In [39]:
do(9, 251, 898)

[251, 898]

# Day 10: Elves Look, Elves Say
1. Look and Say 40
2. Look and Say 50

In [40]:
in10: str = "3113322113"

In [49]:
def day10_1(word, n=40):
    """ Look and Say 40 """
    def compute(word):
        for letter, elements in groupby(word):
            yield str(sum(1 for _ in elements))
            yield letter

    for _ in range(n):
        word = compute(word)

    return sum(1 for _ in word)

In [52]:
def day10_2(word):
    """ Look and Say 50 """
    return day10_1(word, 50)

In [53]:
do(10, 329356, 4666278)

[329356, 4666278]

# Day 11: Corporate Policy
1. New password
2. New new password

In [82]:
in11: str = "hepxcrrq"

In [85]:
def day11_1(password):
    """ New password """
    ord_a = ord("a")
    bad_letters = {ord(letter) - ord_a for letter in "iol"}
    password_int = [ord(char) - ord_a for char in password]
    while True:
#        print("".join(chr(i + ord_a) for i in password_int))
        # increment
        index = len(password_int) - 1
        while True:
            d, m = divmod(password_int[index] + 1, 26)
            if m in bad_letters: # rule 2
                m += 1
            password_int[index] = m
            if d == 0:
                break
            index -= 1

        # check validity
        if len({ # rule 3
                i
                for i,j in zip(password_int, password_int[1:])
                if i == j
        }) < 2:
            continue
        if not any( # rule 1
                i+1==j and j+1==k
                for i,j,k in zip(password_int, password_int[1:], password_int[2:])
        ):
            continue

        # valid
        break

    return "".join(chr(i + ord_a) for i in password_int)

In [88]:
def day11_2(password):
    """ New new password """
    new_password = day11_1(password)
    return day11_1(new_password)

In [90]:
do(11, "hepxxyzz", "heqaabcc")

['hepxxyzz', 'heqaabcc']

# Day 12: JSAbacusFramework.io
1. Sum integer in the json
2. Sum integer except "red" dict in the json

In [102]:
in12: List = data(12, json.loads)

In [113]:
def day12_1(json, avoid=None):
    """ Sum integer in the json """
    if isinstance(json, int):
        return json
    if isinstance(json, str):
        return 0
    if isinstance(json, dict):
        if avoid is not None and avoid in json.values():
            return 0
        return sum(day12_1(value, avoid) for value in json.values())
    if isinstance(json, list):
        return sum(day12_1(value, avoid) for value in json)
    raise Exception(json)

In [111]:
def day12_2(json):
    """ Sum integer except "red" dict in the json """
    return day12_1(json, "red")

In [115]:
do(12, 156366, 96852)

[156366, 96852]

# Day 13: Knights of the Dinner Table
1. Find the optimal seating arrangement
2. Find the optimal seating arrangement with me

In [122]:
def happiness_parser(line):
    src, dir, value, dst = re.fullmatch(
        "(.*) would (.*) (.*) happiness units by sitting next to (.*).",
        line,
    ).groups()
    return ((src, dst), int(value) if dir == "gain" else - int(value))

in13: Dict[Tuple[str, str], int] = dict(data(13, happiness_parser))

{('Alice', 'Bob'): 2, ('Alice', 'Carol'): 26, ('Alice', 'David'): -82, ('Alice', 'Eric'): -75, ('Alice', 'Frank'): 42, ('Alice', 'George'): 38, ('Alice', 'Mallory'): 39, ('Bob', 'Alice'): 40, ('Bob', 'Carol'): -61, ('Bob', 'David'): -15, ('Bob', 'Eric'): 63, ('Bob', 'Frank'): 41, ('Bob', 'George'): 30, ('Bob', 'Mallory'): 87, ('Carol', 'Alice'): -35, ('Carol', 'Bob'): -99, ('Carol', 'David'): -51, ('Carol', 'Eric'): 95, ('Carol', 'Frank'): 90, ('Carol', 'George'): -16, ('Carol', 'Mallory'): 94, ('David', 'Alice'): 36, ('David', 'Bob'): -18, ('David', 'Carol'): -65, ('David', 'Eric'): -18, ('David', 'Frank'): -22, ('David', 'George'): 2, ('David', 'Mallory'): 42, ('Eric', 'Alice'): -65, ('Eric', 'Bob'): 24, ('Eric', 'Carol'): 100, ('Eric', 'David'): 51, ('Eric', 'Frank'): 21, ('Eric', 'George'): 55, ('Eric', 'Mallory'): -44, ('Frank', 'Alice'): -48, ('Frank', 'Bob'): 91, ('Frank', 'Carol'): 8, ('Frank', 'David'): -66, ('Frank', 'Eric'): 97, ('Frank', 'George'): -9, ('Frank', 'Mallory'):

In [137]:
def day13_1(hapinesses):
    """ Find the optimal seating arrangement """
    peoples = set(chain(*hapinesses))

    results = {}
    for permutation in permutations(peoples, len(peoples)):
        results[permutation] = sum(
            hapinesses[(src, dst)] + hapinesses[(dst, src)]
            for src, dst in zip(permutation, permutation[1:] + permutation[:1])
        )
    return max(results.values())

In [139]:
def day13_2(hapinesses):
    """ Find the optimal seating arrangement with me """
    peoples = set(chain(*hapinesses))
    for people in peoples:
        hapinesses[(people, "Me")] = 0
        hapinesses[("Me", people)] = 0

    return day13_1(hapinesses)

In [140]:
do(13, 733, 725)

AssertionError: day13_2(in13) got 725; expected None